In [1]:
import numpy as np
import pandas as pd

In [2]:
mock_range = list(pd.date_range('1/1/2011', periods=60, freq='D'))*5
groups = ['a','b','c','d','e']

mock = pd.DataFrame(index=mock_range)
mock = mock.sort_index()
mock['category']=groups * 60
mock['values'] = np.random.randint(0,100,size=300)
mock = mock.reset_index().rename(columns={'index':'date'}).set_index(['date','category'])
# mock

values
date       category        
2011-01-01 a             50
           b             58
           c             83
           d             84
           e              2
2011-01-02 a             74
           b             15
           c             59
           d             95
           e             39
2011-01-03 a             54
           b             33
           c             61
           d             48
           e             55
2011-01-04 a             11
           b             86
           c             19
           d             25
           e             82
2011-01-05 a             64
           b             77
           c             30
           d              8
           e             81
2011-01-06 a             63
           b             20
           c             21
           d             14
           e             55
...                     ...
2011-02-24 a              3
           b             26
           c             75
           d             69
           e             53
2011-02-25 a             14
           b             29
           c             49
           d             10
           e             78
2011-02-26 a             75
           b             68
           c             48
           d              9
           e             87
2011-02-27 a             79
           b             40
           c             57
           d             74
           e             21
2011-02-28 a             10
           b             61
           c             73
           d             99
           e             47
2011-03-01 a             11
           b             41
           c             48
           d              2
           e             56

[300 rows x 1 columns]

In [223]:
len(mock.index.levels[0])

60

In [133]:
# for x in sorted_df.index.levels[0]:
#     print(sorted_df.loc[x])

In [62]:
sorted_df = mock.sort_index(level=0, ascending=False)
# sorted_df

In [64]:
sorted_df.index[0][0]

Timestamp('2011-03-01 00:00:00')

In [66]:
last_window_ix = sorted_df.index[0][0]
last_window_ix

Timestamp('2011-03-01 00:00:00')

In [67]:
last_window = sorted_df.loc[last_window_ix]
last_window

,values
category,
e,56
d,2
c,48
b,41
a,11


In [68]:
last_window.index

Index(['e', 'd', 'c', 'b', 'a'], dtype='object', name='category')

In [79]:
last_window = last_window.sort_index()
last_window

,values
category,
a,11
b,41
c,48
d,2
e,56


In [28]:
for lbl in first_window.index:
    print(lbl)

e
d
c
b
a


In [30]:
len(sorted_df.index.levels)

2

In [32]:
sorted_df.index.names

FrozenList(['date', 'category'])

In [33]:
sorted_df.columns

Index(['values'], dtype='object')

In [36]:
from pandas.api.types import is_numeric_dtype

In [37]:
is_numeric_dtype?

Signature: is_numeric_dtype(arr_or_dtype)
Docstring:
Check whether the provided array or dtype is of a numeric dtype.

Parameters
----------
arr_or_dtype : array-like
    The array or dtype to check.

Returns
-------
boolean : Whether or not the array or dtype is of a numeric dtype.

Examples
--------
>>> is_numeric_dtype(str)
False
>>> is_numeric_dtype(int)
True
>>> is_numeric_dtype(float)
True
>>> is_numeric_dtype(np.uint64)
True
>>> is_numeric_dtype(np.datetime64)
False
>>> is_numeric_dtype(np.timedelta64)
False
>>> is_numeric_dtype(np.array(['a', 'b']))
False
>>> is_numeric_dtype(pd.Series([1, 2]))
True
>>> is_numeric_dtype(pd.Index([1, 2.]))
True
>>> is_numeric_dtype(np.array([], dtype=np.timedelta64))
False
File:      ~/.pyenv/versions/3.6.4/envs/py3/lib/python3.6/site-packages/pandas/core/dtypes/common.py
Type:      function


In [38]:
is_numeric_dtype(sorted_df['values'])

True

In [75]:
new_window = pd.DataFrame(data=[22, 35, 25, 10, 32], columns=['values'], index=list('abcde'))
new_window

,values
a,22
b,35
c,25
d,10
e,32


In [147]:
last_window

,values
category,
a,11
b,41
c,48
d,2
e,56


In [150]:
new_window2 = pd.DataFrame(data=[15, 35, 46, 10, 8], columns=['values'], index=list('abcdf'))
new_window2

,values
a,15
b,35
c,46
d,10
f,8


In [85]:
list(new_window2.index)

['a', 'b', 'c', 'd', 'f']

In [168]:
MIN_VAL = 5

In [169]:
def _pad_windows(*windows):
    cat_union = list(set(v for df in windows for v in df.index))
    patching_df = pd.DataFrame(data=[MIN_VAL]* len(cat_union), index=cat_union)
    results = []
    for df in windows:
        patching_df.columns = df.columns
        results.append(df.combine_first(patching_df))
    return results

In [170]:
_pad_windows(last_window, new_window2)

[   values
 a    11.0
 b    41.0
 c    48.0
 d     2.0
 e    56.0
 f     5.0,    values
 a    15.0
 b    35.0
 c    46.0
 d    10.0
 e     5.0
 f     8.0]

In [171]:
padded_last_window, padded_new_window, padded_new_window2 = _pad_windows(last_window, new_window, new_window2)

In [164]:
padded_new_window2

,values
a,15.0
b,35.0
c,46.0
d,10.0
e,0.0
f,8.0


In [157]:
from scipy.stats import chisquare

In [158]:
new_window2['values'].values

array([15, 35, 46, 10,  8])

In [173]:
padded_new_window

,values
a,22.0
b,35.0
c,25.0
d,10.0
e,32.0
f,5.0


In [174]:
padded_last_window

,values
a,11.0
b,41.0
c,48.0
d,2.0
e,56.0
f,5.0


In [177]:
padded_last_window.iloc[:,0]

a    11.0
b    41.0
c    48.0
d     2.0
e    56.0
f     5.0
Name: values, dtype: float64

In [179]:
res = chisquare(padded_new_window2.iloc[:,0], padded_last_window.iloc[:,0])
res

Power_divergenceResult(statistic=82.66235613979516, pvalue=2.3256371374894756e-16)

In [187]:
padded_last_window[padded_last_window.index != 'a'].sum()

values    152.0
dtype: float64

In [195]:
def _one_v_all_dists(df):
    res = {}
    for cat in df.index:
        val = float(df.loc[cat][0])
        others = float(df[df.index != cat].sum())
        res[cat] = [val, others]
    return res

In [196]:
_one_v_all_dists(padded_last_window)

{'a': [11.0, 152.0],
 'b': [41.0, 122.0],
 'c': [48.0, 115.0],
 'd': [2.0, 161.0],
 'e': [56.0, 107.0],
 'f': [5.0, 158.0]}

In [202]:
p_threhold = 0.001

In [ ]:
from scipy.stats import chisquare

In [203]:
def _detect_trend(obs, exp):
    direction = 1
    if obs[0] < exp[0]:
        direction = -1
    res = chisquare(obs, exp)
    if res[1] < p_threhold:
        return res[1], direction
    return res[1], 0

In [214]:
def _predict(new_windows, last_window):
    padded = _pad_windows(last_window, *new_windows)
    padded_last = padded.pop(0)
    last_1vall = _one_v_all_dists(padded_last)
    res = []
    for new_win in padded:
        new_1vall = _one_v_all_dists(new_win)
        pred = {cat: _detect_trend(new_1vall[cat], last_1vall[cat]) for cat in new_1vall}
        res.append(pd.DataFrame(pred, index=['p', 'direction']).T)
    return res

In [217]:
res = _predict([new_window, new_window2], last_window)
res[1]

,p,direction
a,4.584930e-05,1.0
b,3.629061e-04,-1.0
c,8.596035e-05,-1.0
d,2.841588e-12,1.0
e,7.453623e-12,-1.0
f,7.111260e-05,1.0


In [221]:
res[0].iloc[:,1]

a    1.0
b    0.0
c   -1.0
d    1.0
e   -1.0
f    0.0
Name: direction, dtype: float64